Comparing performance of `INSERT` versus `INSERT OR UPDATE` with a few simple entries, and a database on disk.

In [1]:
import tempfile
import sqlite3
import timeit
import pstats

In [2]:
db_file = tempfile.NamedTemporaryFile()

In [3]:
# from https://www.sqlitetutorial.net/sqlite-python/create-tables/

sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        begin_date text,
                                        end_date text
                                    ); """
 
sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    priority integer,
                                    status_id integer NOT NULL,
                                    project_id integer NOT NULL,
                                    begin_date text NOT NULL,
                                    end_date text NOT NULL,
                                    FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""
print(db_file.name)
conn = sqlite3.connect(db_file.name)
c = conn.cursor()
c.execute(sql_create_projects_table)
c.execute(sql_create_tasks_table)
c.execute("INSERT INTO projects VALUES(1, 'project1', '', '')")
c.close()
conn.commit()

/tmp/tmpt740hjyi


In [4]:
def prepare_tables():
    c = conn.cursor()
    c.execute("DELETE FROM tasks")
    for i in range(1, 2000):
        if i % 2 == 0 or i % 3 or i % 5:
            c.execute(f"INSERT INTO tasks VALUES({i}, 'task{i}', {i}, {i}, '', '', 1)")
    c.close()
    conn.commit()

First scenario, we have already multiples of 2, 3, and 5 inserted. Then we `INSERT OR REPLACE` everything from 1 to 2000.

In [5]:
prepare_tables()
start = timeit.default_timer()

c = conn.cursor()
for i in range(1, 2000):
    c.execute(f"INSERT OR REPLACE INTO tasks VALUES({i}, 'task{i}', {i}, {i}, '', '', 1)")
c.close()
conn.commit()

end = timeit.default_timer()
print(end - start)

0.040296187999047106


Second scenario. Same as before, but we skip the values we know are already in the DB, and just `INSERT`. Obviously faster.

In [6]:
prepare_tables()
start = timeit.default_timer()

c = conn.cursor()
for i in range(1, 2000):
    if i % 2 == 0 or i % 3 or i % 5:
        continue
    c.execute(f"INSERT INTO tasks VALUES({i}, 'task{i}', {i}, {i}, '', '', 1)")
c.close()
conn.commit()

end = timeit.default_timer()
print(end - start)

0.01291432200014242


Final scenario, just to confirm how long it takes to insert all tasks.

In [7]:
c = conn.cursor()
c.execute("DELETE FROM tasks")
c.close()
conn.commit()
start = timeit.default_timer()

c = conn.cursor()
for i in range(1, 2000):
    c.execute(f"INSERT INTO tasks VALUES({i}, 'task{i}', {i}, {i}, '', '', 1)")
c.close()
conn.commit()

end = timeit.default_timer()
print(end - start)

0.03846671400060586


In [8]:
conn.close()
db_file.close()

It took 1/3 of the time of `INSERT OR UPDATE`, when doing a simple `INSERT` and skipping the items we knew existed.

The number of existing entries affects how much improvement we have. But clearly `INSERT` performs better than `INSERT OR UPDATE`. If there is no hard-requirement on using `INSERT OR UPDATE`, replacing it by `INSERT` will improve the performance.